In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Hàm tải và chuẩn bị dữ liệu từ thư mục ảnh
def load_data(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):
            label = filename.split("_")[0]
            if label in ["right", "forward", "left"]:
                img = cv2.imread(os.path.join(data_dir, filename))
                img = cv2.resize(img, (224, 224))  # Resize về kích thước 224x224
                images.append(img)
                labels.append(["right", "forward", "left"].index(label))
    return np.array(images), np.array(labels)

# Tải dữ liệu và chia tập huấn luyện/validation
data_dir = "balanced_data"  # Thay bằng đường dẫn thực tế đến thư mục chứa ảnh
images, labels = load_data(data_dir)
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu (MobileNetV2 yêu cầu giá trị pixel trong [-1, 1])
train_images = tf.keras.applications.mobilenet_v2.preprocess_input(train_images)
val_images = tf.keras.applications.mobilenet_v2.preprocess_input(val_images)

# Định nghĩa mô hình với MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Không huấn luyện lại MobileNetV2

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation="softmax")  # 3 lớp đầu ra: right, forward, left
])

# Compile mô hình
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Huấn luyện mô hình
history = model.fit(train_images, train_labels, epochs=15, validation_data=(val_images, val_labels))

# Vẽ đồ thị loss
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("loss_plot_clearn.png")  # Lưu đồ thị thành file
plt.close()

# Dự đoán trên tập validation
val_predictions = model.predict(val_images)
val_predictions = np.argmax(val_predictions, axis=1)  # Chuyển từ xác suất sang nhãn

# Tính confusion matrix
cm = confusion_matrix(val_labels, val_predictions)

# Vẽ confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=["right", "forward", "left"], 
            yticklabels=["right", "forward", "left"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix on Validation Set")
plt.savefig("confusion_matrix_clean.png")  # Lưu confusion matrix thành file
plt.close()

# Lưu mô hình dưới dạng .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("steering_model_balanced.tflite", "wb") as f:
    f.write(tflite_model)


Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 24s 725ms/step - accuracy: 0.5674 - loss: 0.9365 - val_accuracy: 0.8093 - val_loss: 0.5031
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 681ms/step - accuracy: 0.8166 - loss: 0.4810 - val_accuracy: 0.9070 - val_loss: 0.3825
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 643ms/step - accuracy: 0.9106 - loss: 0.3234 - val_accuracy: 0.9302 - val_loss: 0.3192
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 681ms/step - accuracy: 0.9333 - loss: 0.2557 - val_accuracy: 0.9302 - val_loss: 0.2897
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 672ms/step - accuracy: 0.9480 - loss: 0.2173 - val_accuracy: 0.9442 - val_loss: 0.2676
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 650ms/step - accuracy: 0.9321 - loss: 0.2016 - val_accuracy: 0.9349 - val_loss: 0.2592
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 652ms/step - accuracy: 0.9549 - loss: 0.1698 - val_accuracy: 0.9442 - val_loss: 0.2479
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 645ms/step - accuracy: 0.9584 - loss: 0.1528 - val_accu

INFO:tensorflow:Assets written to: C:\Users\nguye\AppData\Local\Temp\tmpt8i7xuud\assets


Saved artifact at 'C:\Users\nguye\AppData\Local\Temp\tmpt8i7xuud'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_312')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2065122815216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065122813280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065122816272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065122808352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065122811168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065122818736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065124542592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065124540832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065122818032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065124542944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2

In [5]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Dự đoán trên tập validation
val_predictions = model.predict(val_images)
val_predictions = np.argmax(val_predictions, axis=1)  # Chuyển từ xác suất sang nhãn

# Tính confusion matrix
cm = confusion_matrix(val_labels, val_predictions)

# Vẽ confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=["right", "forward", "left"], 
            yticklabels=["right", "forward", "left"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix on Validation Set")
plt.savefig("confusion_matrix_clean.png")  # Lưu confusion matrix thành file
plt.close()

7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 494ms/step


In [6]:

# # Tải mô hình .tflite
# interpreter = tf.lite.Interpreter(model_path="steering_model_clean.tflite")
# interpreter.allocate_tensors()

# # Lấy thông tin input và output của mô hình
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# # Kiểm tra trên luồng video
# stream_url = "http://192.168.46.130:81/stream"
# cap = cv2.VideoCapture(stream_url)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Không thể đọc luồng video.")
#         break

#     # Chuẩn bị frame cho dự đoán
#     frame_resized = cv2.resize(frame, (224, 224))
#     frame_processed = tf.keras.applications.mobilenet_v2.preprocess_input(frame_resized)
#     input_data = np.expand_dims(frame_processed, axis=0).astype(np.float32)

#     # Đặt dữ liệu đầu vào và chạy dự đoán
#     interpreter.set_tensor(input_details[0]["index"], input_data)
#     interpreter.invoke()

#     # Lấy kết quả dự đoán
#     output_data = interpreter.get_tensor(output_details[0]["index"])
#     prediction = np.argmax(output_data[0])
#     result = ["right", "forward", "left"][prediction]
#     print(f"Dự đoán: {result}")

#     # Hiển thị frame (tùy chọn)
#     cv2.imshow("Video Stream", frame)
#     if cv2.waitKey(1) & 0xFF == ord("q"):  # Nhấn 'q' để thoát
#         break

# # Giải phóng tài nguyên
# cap.release()
# cv2.destroyAllWindows()